##### What is this notebook about?

- We’ll create an example agent from scratch using ReAct pattern:
    - Reason
    - Act
    - Observe
    - Repeat

In [1]:
import openai
import re
import httpx
import os

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

In [2]:
# Load the AI model (LLM)

_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.getenv('sk')

client = OpenAI(
  api_key=api_key,
)

In [3]:
# Test the AI model 

if False:

    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Hello world"}]
    )
    print("Raw chat completion")
    print(chat_completion.choices[0].message.content)

In [4]:
# Define Agent

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o-mini", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [5]:
# Define tools

def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [6]:
# Define the agent's task in ReAct pattern

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [7]:
# Create the agent

abot = Agent(prompt)

### Run the agent step-by-step - Example Task 1:

In [8]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action. 
Action: average_dog_weight: Toy Poodle
PAUSE


In [9]:
result = average_dog_weight("Toy Poodle")

In [10]:
result

'a toy poodles average weight is 7 lbs'

In [11]:
next_prompt = "Observation: {}".format(result)

In [12]:
abot(next_prompt)

'Answer: A Toy Poodle weighs 7 lbs.'

In [13]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a

In [14]:
abot = Agent(prompt)

### Run the agent step-by-step - Example Task 2:

In [15]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier, then add those weights together to get the combined weight of the two dogs. \nAction: average_dog_weight: Border Collie\nPAUSE'

In [16]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [17]:
abot(next_prompt)

'Thought: Now I need to find the average weight of a Scottish Terrier to complete the calculation for the combined weight of both dogs. \nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [18]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [19]:
abot(next_prompt)

'Thought: I have the average weights for both dogs: a Border Collie weighs 37 lbs and a Scottish Terrier weighs 20 lbs. Now I will add these two weights together to find their combined weight. \nAction: calculate: 37 + 20\nPAUSE'

In [20]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [21]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

### Run the agent in single shot (i.e. loop on ReAct pattern): 

In [22]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [23]:
def query(question, max_turns=5):

    i = 0

    # Create the agent
    bot = Agent(prompt)
    
    # Initialize the prompt with the task
    next_prompt = question
    
    while i < max_turns:
        
        i += 1
        
        # Execute the AI model with latest prompt
        result = bot(next_prompt)
        print(result)

        # Search if AI model wants to execute an action
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        
        if actions:

            print(f"--> found actions regexp match: {actions}")

            # There is an action to run - get the action
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            
            # Execute the action
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            
            # Get the observation & append it into the prompt
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)

        else:

            # No action to run. End of task
            return

In [24]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. 
Action: average_dog_weight: Border Collie
PAUSE
--> found actions regexp match: [<re.Match object; span=(0, 41), match='Action: average_dog_weight: Border Collie'>]
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight. 
Action: average_dog_weight: Scottish Terrier
PAUSE
--> found actions regexp match: [<re.Match object; span=(0, 44), match='Action: average_dog_weight: Scottish Terrier'>]
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights for both dogs: 37 lbs for the Border Collie and 20 lbs for the Scottish Terrier. I can calculate their combined weight by adding these two values together. 
Action: calculate: 3

### References:

> https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/  
> https://til.simonwillison.net/llms/python-react-pattern


